In [ ]:
! pip install gym
! git clone https://github.com/gyyang/neurogym.git
%cd neurogym/
! pip install -e .
! pip install tasks


In [ ]:
!pip install shimmy

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()
run=wandb.init()

In [ ]:
# Environment
task = 'DawTwoStep-v0'
kwargs = {'dt': 100}
seq_len = 100
env = ngym.make(task, **kwargs)
ob_size = env.observation_space.shape[0]
act_size = env.action_space.n

In [10]:
from re import template
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import neurogym as ngym
from tqdm import tqdm


alpha=0.1
gamma=0.9
epsilon=1.0
_ = env.reset()


class Agent:
    def __init__(self):
        self.Q = np.zeros([ob_size,act_size])

    def selectAction(self, cortex_state):
        ob = np.argmax(cortex_state)
        if np.max(self.Q[ob,:]) == 0 or np.random.rand() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(self.Q[ob,:])
        return action

    def updateQ(self, cortex_state, action, reward, next_cortex_state, done):
        ob, new_ob= np.argmax(cortex_state), np.argmax(next_cortex_state)
        next_action = np.argmax(self.Q[new_ob,:])
        scalar_rpe = reward + (gamma*self.Q[new_ob, next_action])-self.Q[ob, action]
        self.Q[ob, action] += alpha*scalar_rpe

agent = Agent()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
episodes = 2000
trial = env.new_trial()
episode_max_length= 100
for i in tqdm(range(episodes)):
    total_reward = 0
    cortex_state = env.reset()
    d = False
    for t in range(episode_max_length):
        action = agent.selectAction(cortex_state)
        next_cortex_state, reward, d, _ = env.step(action)
        agent.updateQ(cortex_state, action, reward, next_cortex_state, d)
        cortex_state = next_cortex_state
        total_reward += reward
        if d == True:
            break
env.close()

100%|██████████| 2000/2000 [00:09<00:00, 201.53it/s]
